#  Implementação de um Agente em Flappy Bird

### Imports

In [44]:
import gym
from gym import spaces
import gym_ple
import numpy as np

## Flappy Bird Sem Visão Computacional

### Ambiente

Modelagem do ambiente do Flappy Bird usando o GameState como observação.

#### Observação:
 * Posição Y do pássaro.
 * Velocidade Y do pássaro.
 * Distância do pássaro até o próximo cano.
 * Posição Y da parte de cima do próximo cano.
 * Posição Y da parte de baixo do próximo cano.
 * Distância do pássaro até o cano depois do próximo cano.
 * Posição Y da parte de cima do cano depois do próximo cano.
 * Posição Y da parte de baixo do cano depois do próximo cano.

In [51]:
class FlappyBirdEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array']}

  def __init__(self):
    self.n_pipes = 0
    self.env = gym.make("FlappyBird-v0")
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions:
    self.action_space = self.env.action_space
    # Example for using image as input:
    self.observation_space = spaces.Box(low=np.array([0, -10.0, 0, 0, 0, 0, 0, 0]),
                                        high=np.array([512, 10.0, 588.0, 512, 512, 588.0, 512, 512]),
                                        dtype=np.float32)

  def step(self, action):
    observation, reward, done, info = self.env.step(action)
    observation = np.array(list(self.env.game_state.getGameState().values()))
    if reward > 0:
        self.n_pipes += 1
    if done:
        reward = -1
    reward += 0.1
    reward += (75 - abs(observation[0] - (observation[3] + observation[4])/2))*max((300 - observation[2])/300, 0)/750
    
    if self.n_pipes >= 500:
        done = True
    
    return observation, reward, done, info

  def reset(self):
    self.env.reset()
    self.n_pipes = 0
    observation = np.array(list(self.env.game_state.getGameState().values()))
    return observation  # reward, done, info can't be included

  def render(self, mode='human'):
    return self.env.render(mode=mode)
    
  def close (self):
    self.env.close()

Checando se o nosso ambiente satisfaz as propriedades do Gym.

In [52]:
from stable_baselines.common.env_checker import check_env

env = FlappyBirdEnv()
check_env(env)

Criando um vetor com 16 ambientes para multiprocessamento.

In [53]:
from stable_baselines.common import set_global_seeds, make_vec_env
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
import os.path

env = make_vec_env(FlappyBirdEnv, n_envs = 8)

if os.path.isfile('VecNormalize.pkl'):
    env = VecNormalize.load("VecNormalize.pkl", env)
    print ("Importing Environment")
else:
    env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)
    env.save("VecNormalize.pkl")
    print("Normalizing Environment")

Importing Environment


### Agente

Criando o modelo de PPO2 com a biblioteca Stable Baselines.

In [5]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import PPO2

model = PPO2(MlpPolicy, env, n_steps = 1024, learning_rate=0.0001, nminibatches = 64, lam = 0.95, gamma = 0.99, noptepochs= 15, ent_coef= 0.0, cliprange= 0.2, verbose=1)





Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Você tem a opção de usar um modelo pré-treinado:

In [54]:
from stable_baselines import PPO2

model = PPO2.load("trained_models/FrozenEnvPPO2")
model.set_env(env)

Loading a model without an environment, this model cannot be trained until it has a valid environment.


Criando um Callback

In [68]:
from stable_baselines.common.callbacks import EvalCallback

EvalEnv = make_vec_env(FlappyBirdEnv, n_envs = 1)

if os.path.isfile('VecNormalize.pkl'):
    EvalNormEnv = VecNormalize.load("VecNormalize.pkl", EvalEnv)
    print ("Importing Environment")
else:
    EvalNormEnv = VecNormalize(EvalEnv, norm_obs=True, norm_reward=False, clip_obs=10.)
    print("Normalizing Environment")

EvalNormEnv.training = False
    
eval_callback = EvalCallback(EvalNormEnv, best_model_save_path='./eval_models/',
                             log_path='./logs/', n_eval_episodes = 10,
                             eval_freq=1024,deterministic=True, render=False)

Importing Environment


In [70]:
# TODO - Callback do VecNormalize.save()

#### Treinamento do Modelo:

In [69]:
if os.path.isfile('VecNormalize.pkl'):
    EvalNormEnv = VecNormalize.load("VecNormalize.pkl", EvalEnv)
    EvalNormEnv.training = False
    eval_callback.eval_env = EvalNormEnv

model.learn(total_timesteps=16384, callback=eval_callback)
# model.learn(total_timesteps=65536, callback=eval_callback)
env.save("VecNormalize.pkl")

Eval num_timesteps=8192, episode_reward=653.89 +/- 496.08
Episode length: 3517.80 +/- 2638.20
New best mean reward!
--------------------------------------
| approxkl           | 0.0012977485  |
| clipfrac           | 0.011995443   |
| ep_len_mean        | 688           |
| ep_reward_mean     | 124           |
| explained_variance | 0.0832        |
| fps                | 30            |
| n_updates          | 1             |
| policy_entropy     | 0.30509627    |
| policy_loss        | -0.0010997339 |
| serial_timesteps   | 1024          |
| time_elapsed       | 0             |
| total_timesteps    | 8192          |
| value_loss         | 2.7651675     |
--------------------------------------
Eval num_timesteps=16384, episode_reward=775.90 +/- 551.30
Episode length: 4187.90 +/- 2950.30
New best mean reward!
--------------------------------------
| approxkl           | 0.0020303985  |
| clipfrac           | 0.023852538   |
| ep_len_mean        | 755           |
| ep_reward_mean     | 136

#### Testando o Modelo: 

In [71]:
TestEnv = make_vec_env(FlappyBirdEnv, n_envs = 1)

if os.path.isfile('VecNormalize.pkl'):
    TestNormEnv = VecNormalize.load("VecNormalize.pkl", TestEnv)
else:
    TestNormEnv = VecNormalize(TestEnv, norm_obs=True, norm_reward=False, clip_obs=10.)
    
TestNormEnv.training = False

In [72]:
import itertools

obs = TestNormEnv.reset()

for t in itertools.count():
    TestNormEnv.render()
    action, _states = model.predict(obs)
    obs, rewards, dones, info = TestNormEnv.step(action)
    if dones:
        break

print(t)
# print(TestNormEnv.get_attr('n_pipes'))
TestNormEnv.close()

6918


#### Salvando o Modelo:

In [73]:
model.save("trained_models/FrozenEnvPPO2")

#### Gravando um Episódio: (Not Working)

Criando o ambiente gravado.

In [88]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

video_folder = 'videos/'
video_length = 100

obs = EvalEnv.reset()

VideoEnv = VecVideoRecorder(EvalEnv, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="FlappyBird")

Rodando o episódio.

In [92]:
images = []

obs = EvalEnv.reset()
img = EvalEnv.render(mode='rgb_array')

for t in itertools.count():
    images.append(img)
    EvalEnv.render()
    action, _ = model.predict(obs)
    obs, _, dones, _ = EvalEnv.step(action)
    img = EvalEnv.render(mode='rgb_array')
    if dones:
        break

EvalEnv.close()

imageio.mimsave('lander_a2c.gif', images, fps=29)

## Flappy Bird com Visão Computacional

A implementar

In [71]:
ENV_NAME = "FlappyBird-v0"

from stable_baselines.common.vec_env import VecFrameStack
from stable_baselines.common import set_global_seeds, make_vec_env
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize

env_vision = make_vec_env("FlappyBird-v0", n_envs = 8)
env_vision = VecFrameStack(env_vision, n_stack=4)

In [74]:
from stable_baselines import PPO2
from stable_baselines.common.policies import CnnPolicy

modelVision = PPO2(CnnPolicy, env_vision, n_steps = 128, nminibatches = 4, lam = 0.98, gamma = 0.999, noptepochs= 8, ent_coef= 0.01, verbose=1)

In [75]:
modelVision.learn(total_timesteps=10000)

ResourceExhaustedError: OOM when allocating tensor of shape [512,288,12] and type float
	 [[node input/sub/y (defined at D:\ProgramData\Anaconda3\lib\site-packages\stable_baselines\common\input.py:33) ]]

Original stack trace for 'input/sub/y':
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "D:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
    self.asyncio_loop.run_forever()
  File "D:\ProgramData\Anaconda3\lib\a

In [ ]:
obs = env_vision.reset()
dones = 0
while not dones:
    action, _states = modelVision.predict(obs)
    obs, rewards, dones, info = env_vision.step(action)
    env.render()

env_vision.close()